CTM evaluation

In [ ]:
import pandas as pd
import numpy as np


from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
# from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords

import nltk

import os
from pathlib import Path
import json
from datetime import datetime
import pickle

os.environ["TOKENIZERS_PARALLELISM"] = "false"          # disable huggingface warning

In [ ]:
%load_ext autoreload

In [ ]:
import sys

sys.path.append('../')

In [ ]:
# load the dataset
# TODO: load external dataset

%autoreload 2
from dataset_loader import GENRES, load_dataset

genre = GENRES.INDIE
dataset = load_dataset(genre)

dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 725737 entries, 25636 to 4179608
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   index         725737 non-null  int64 
 1   app_id        725737 non-null  int64 
 2   app_name      725737 non-null  object
 3   review_text   725737 non-null  object
 4   review_score  725737 non-null  int64 
 5   review_votes  725737 non-null  int64 
 6   genre_id      725737 non-null  object
 7   category_id   725737 non-null  object
dtypes: int64(4), object(4)
memory usage: 49.8+ MB


In [ ]:
# data preprocessing
# MUST BE IDENTICAL TO THE ONE USED IN TRAINING

import sys
sys.path.append('../../sa/')

%autoreload 2
import str_cleaning_functions

# copied from lda_demo_gridsearch.ipynb
def cleaning(df, review):
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links2(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.clean(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.deEmojify(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_non_letters(x))
    df[review] = df[review].apply(lambda x: x.lower())
    df[review] = df[review].apply(lambda x: str_cleaning_functions.unify_whitespaces(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_stopword(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.unify_whitespaces(x))

# def cleaning_strlist(str_list):
#     str_list = list(map(lambda x: clean(x), str_list))
#     str_list = list(map(lambda x: deEmojify(x), str_list))

#     str_list = list(map(lambda x: x.lower(), str_list))
#     str_list = list(map(lambda x: remove_num(x), str_list))
#     str_list = list(map(lambda x: unify_whitespaces(x), str_list))

#     str_list = list(map(lambda x: _deaccent(x), str_list))
#     str_list = list(map(lambda x: remove_non_alphabets(x), str_list))
#     str_list = list(map(lambda x: remove_stopword(x), str_list))
#     return str_list

# copied from bert_demo_gridsearch.ipynb
def cleaning_little(df, review):
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links2(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.clean(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.deEmojify(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.unify_whitespaces(x))

In [ ]:
# create a copy of the dataset, as we need both untouched text and cleaned text

dataset_preprocessed = dataset.copy()

In [ ]:
cleaning(dataset_preprocessed, 'review_text')
cleaning_little(dataset, 'review_text')

In [ ]:
X_preprocessed_temp = dataset_preprocessed['review_text'].values
X_temp = dataset['review_text'].values

In [ ]:
assert X_temp.shape == X_preprocessed_temp.shape, "X_temp and X_preprocessed_temp should have the same shape. Found: {} and {}".format(X_temp.shape, X_preprocessed_temp.shape)
assert len(X_temp) == len(X_preprocessed_temp), "X_temp and X_preprocessed_temp should have the same length. Found: {} and {}".format(len(X_temp), len(X_preprocessed_temp))

In [ ]:
# remove docs with 0 len

X, X_preprocessed = [], []

for i, (doc, doc_preprocessed) in enumerate(zip(list(X_temp), list(X_preprocessed_temp))):
    if len(doc) == 0 or len(doc_preprocessed) == 0:
        continue

    X.append(doc)
    X_preprocessed.append(doc_preprocessed)

In [ ]:
assert len(X) == len(X_preprocessed), "X and X_preprocessed should have the same length. Found: {} and {}".format(len(X), len(X_preprocessed))

(737139, 737139)

Apply lemmatizing to the preprocessed dataset as well (for BoW)

The function is identical in LDA

In [ ]:
# do lemmatization, but not stemming (as part of speech is important in topic modelling)
# use nltk wordnet for lemmatization

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemma = WordNetLemmatizer()

# from https://stackoverflow.com/questions/25534214/nltk-wordnet-lemmatizer-shouldnt-it-lemmatize-all-inflections-of-a-word

# from: https://www.cnblogs.com/jclian91/p/9898511.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None     # if none -> created as noun by wordnet
    
def lemmatization(text):
   # use nltk to get PoS tag
    tagged = nltk.pos_tag(nltk.word_tokenize(text))

    # then we only need adj, adv, verb, noun
    # convert from nltk Penn Treebank tag to wordnet tag
    wn_tagged = list(map(lambda x: (x[0], get_wordnet_pos(x[1])), tagged))

    # lemmatize by the PoS
    lemmatized = list(map(lambda x: lemma.lemmatize(x[0], pos=x[1] if x[1] else wordnet.NOUN), wn_tagged))
    # lemma.lemmatize(wn_tagged[0], pos=wordnet.NOUN)

    return lemmatized

In [ ]:
X_preprocessed = list(map(lambda x: lemmatization(x), X_preprocessed))
X_preprocessed = list(map(lambda x: ' '.join(x), X_preprocessed))

---

Load the training result

In [ ]:
sys.path.append('../')

from eval_metrics import compute_inverted_rbo, compute_topic_diversity, compute_pairwise_jaccard_similarity, \
                        METRICS, SEARCH_BEHAVIOUR, COHERENCE_MODEL_METRICS

In [ ]:
from ctm_dataset_creation import create_ctm_dataset
from ctm_utils import _load_ctm_model, _get_topics, _get_topic_document_metrix, _get_topic_word_metrix

In [ ]:
# load the model from disk to compare the results

search_behaviour = SEARCH_BEHAVIOUR.GRID_SEARCH
training_datetime = datetime(2024, 1, 29, 21, 29, 10)
training_folder = Path(f'ctm_{search_behaviour.value}_{training_datetime.strftime("%Y%m%d_%H%M%S")}')

training_result_json_path = training_folder.joinpath('result.json')
with open(training_result_json_path, 'r') as f:
    training_result = json.load(f)


# load the embeddings
model_name_or_path = training_result['best_hyperparameters']['sbert_params']['model_name_or_path']
embeddings_path = training_folder.joinpath(f'embeddings_{model_name_or_path}.pkl')
with open(embeddings_path, 'rb') as f:
    embeddings = np.load(f)

best_model_path = training_result['best_model_checkpoint']
ctm_hyperparameters = training_result['best_hyperparameters']['ctm_params']
sbert_params = training_result['best_hyperparameters']['sbert_params']

# ctm_hyperparameters['bow_size'] = 2000
# ctm_hyperparameters['contextual_size'] = 768

best_model = _load_ctm_model(Path(best_model_path), ctm_hyperparameters)

# create the dataset on the fly
vectorizer = pickle.load(open(Path(best_model_path).joinpath('count_vectorizer.pkl'), 'rb'))

training_dataset, _, _, _ = create_ctm_dataset(
    X, X_preprocessed, 
    sbert_params, training_folder,
    vectorizer=vectorizer)

---

Visualization

with the same set of training dataset, we can visualize its topic modeling performance

Create a pyLDAVis alike graph

In [ ]:
import pyLDAvis as vis

vocab = vectorizer.get_feature_names_out()

lda_vis_data = best_model.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

In [ ]:
# get top N keywords for each topic

topic_list = best_model.get_topic_lists(10)

topic_list

In [ ]:
doc_topic_distribution = best_model.get_doc_topic_distribution(training_dataset, n_samples=20)

top_docs = best_model.get_top_documents_per_topic_id(X, doc_topic_distribution, 8, k=10)

In [ ]:
for tt in [t[0] for t in top_docs]:
    print(tt)

LEGIT THE BEST GAME EVER BETTER THAN MINECRAFT, SO MANY THINGS TO DO!!!!!!! BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!
Just realized I've had this game for years and never reviewed it. Which is just horrible of me. Of all the games in my steam library this deserves a review. I've enjoyed this 

In [ ]:
# within the topic lists (the words)
# find out common words between topics

from itertools import combinations

topic_list = best_model.get_topic_lists(k=10)

common_words = set()
for topic1, topic2 in combinations(topic_list, 2):
    common_words.update(set(topic1).intersection(set(topic2)))

common_words = list(common_words)
common_words.sort()
common_words

['boss',
 'buy',
 'check',
 'content',
 'course',
 'explore',
 'felt',
 'friend',
 'fun',
 'game',
 'get',
 'great',
 'hour',
 'item',
 'like',
 'list',
 'love',
 'mention',
 'minecraft',
 'new',
 'number',
 'one',
 'play',
 'recommend',
 'say',
 'special',
 'spoil',
 'still',
 'stuff',
 'terrarium',
 'time',
 'update',
 'well',
 'world']